In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import sys
sys.path.append('../')
import tsregressor

In [8]:
df = pd.read_csv('air_passenger.csv',sep=';')
df['date'] = pd.to_datetime(df['date'],format="%d/%m/%Y")

# noisy inputs
df['exog1'] = df['volume'] + np.random.rand(len(df.index))*100
df['exog2'] = df['volume'] ** .5
df['exog3'] = df['volume'] + np.random.rand(len(df.index))*100

print(df.shape)
df.head(2)

(144, 5)


,date,volume,exog1,exog2,exog3
0,1949-01-01,112,179.930077,10.583005,138.858567
1,1949-02-01,118,158.528220,10.862780,151.129820


In [23]:
fig = px.line(df,x='date',y='volume')
fig.show()

In [24]:
fig = px.line(df,x='date',y='exog3')
fig.show()

In [22]:
horizon = 24
test_date = pd.date_range(start=df['date'].min(),periods=len(df.index)-horizon+1,freq=df['date'].dt.freq).max()
df_temp = df.loc[df['date']<test_date]

clf = tsregressor.TSRegressor(horizon=horizon,
                              exog=['exog1','exog2'],
                              target='volume',
                              date=df_temp['date'])
#pred = clf.fit_predict(df,df['volume'])

clf.fit(df_temp,df_temp['volume'])
df_trans = clf.transform(df_temp)
pred = clf.predict(df_trans)

#plt.plot(df['date'],df['volume'],label='y')
fig = px.line(df,x='date',y=['volume',pred])#,label='y')
#px.line(df,x='date',y=pred)#,label=pred)
#plt.plot(pred,label='pred')
mae = np.mean(abs(df.loc[df['date']>test_date,'volume'].values - pred.loc[pred.index>test_date].values))
#plt.title('forecast MAE={:0.2f}'.format(mae))
#plt.legend()
#plt.show()
fig.update_layout(title={'text':'Volume Prediction vs Test data (MAE={:0.2f})'.format(mae)})
fig.show()

In [6]:
df_feat = pd.DataFrame({'feat':clf.feat_cols,'importance':clf.clf.feature_importances_})
df_feat.sort_values('importance').tail(20)

,feat,importance
24,exog1_Lag43,0.005710
64,target_trans_Lag35,0.006963
55,target_trans_Lag26,0.007327
37,exog2_Lag32,0.007332
43,exog2_Lag38,0.007472
53,target_trans_Lag24,0.007939
34,exog2_Lag29,0.008848
60,target_trans_Lag31,0.009177
14,exog1_Lag33,0.009458
66,target_trans_Lag37,0.009633
